1. Install and Import Libraries

In [2]:
# Install required libraries
!pip install geopandas shapely openpyxl folium

# Import libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✓ Environment ready!")

✓ Environment ready!


2. Upload Statistical Area Files

In [3]:
print("Please upload ALL 6 shapefile files (statistical_areas_2022.*):")
uploaded_areas = files.upload()

# Load the shapefile
gdf_areas = gpd.read_file('statistical_areas_2022.shp')
print(f"✓ Loaded {len(gdf_areas)} statistical areas.")

Please upload ALL 6 shapefile files (statistical_areas_2022.*):


Saving statistical_areas_2022.cpg to statistical_areas_2022.cpg
Saving statistical_areas_2022.dbf to statistical_areas_2022.dbf
Saving statistical_areas_2022.prj to statistical_areas_2022.prj
Saving statistical_areas_2022.shp to statistical_areas_2022.shp
Saving statistical_areas_2022.shx to statistical_areas_2022.shx
Saving statistical_areas_2022.xml to statistical_areas_2022.xml
✓ Loaded 25 statistical areas.


3. Upload the Matnasim File

In [5]:
print("Please upload your Matnasim Excel file (Eilat_matnasim.xlsx):")
uploaded_data = files.upload()
filename = list(uploaded_data.keys())[0]

# Use read_excel for .xlsx files
df = pd.read_excel(filename)

print(f"✓ Loaded {len(df)} community centers.")
print(f"Columns found: {df.columns.tolist()}")

Please upload your Matnasim Excel file (Eilat_matnasim.xlsx):


Saving Eilat_matnasim.xlsx to Eilat_matnasim (1).xlsx
✓ Loaded 8 community centers.
Columns found: ['matnas_name', 'full_address', 'person_in_charge', 'phone_number', 'activity_days', 'Facility_area', 'occupancy', 'number_of_activity_rooms', 'shelter_and_where', 'stat_2022', 'semel_yish', 'location_lat', 'location_lng']


4. Perform the Spatial Join

In [6]:
# 1. Create points from coordinates
# Using your exact column names: loaction_lng and location_lat
geometry = [Point(xy) for xy in zip(df['location_lng'], df['location_lat'])]
gdf_matnasim = gpd.GeoDataFrame(df, geometry=geometry)

# 2. Set coordinate system to standard Lat/Lng (WGS84)
gdf_matnasim.set_crs(epsg=4326, inplace=True)

# 3. Match the coordinate system of the map (Israel TM Grid)
gdf_matnasim = gdf_matnasim.to_crs(gdf_areas.crs)

# 4. Perform the spatial join to find the area for each point
# This matches each matnas to a 'STAT_2022' code
result = gpd.sjoin(gdf_matnasim, gdf_areas[['STAT_2022', 'geometry']], how='left', predicate='within')

# 5. Update the 'stat_2022' column with the results
result['stat_2022'] = result['STAT_2022']

# Clean up temporary columns
final_df = result.drop(columns=['geometry', 'index_right', 'STAT_2022'])

print("✓ Statistical areas (stat_2022) have been filled!")
print(final_df[['matnas_name', 'stat_2022']].head())

✓ Statistical areas (stat_2022) have been filled!
                matnas_name  stat_2022
0  מתנ"ס שחמון מתחם גיל הרך         32
1              מתנ"ס קולייר         21
2               מתנ"ס יעלים         14
3           מתנ"ס בית רובין         16
4                    ספרייה         16


5. Save and Download Results

In [7]:
# Save to Excel and CSV
final_df.to_excel('Eilat_matnasim_with_stat_2022.xlsx', index=False)
final_df.to_csv('Eilat_matnasim_with_stat_2022.csv', index=False)

print("✓ Files saved. Downloading now...")
files.download('Eilat_matnasim_with_stat_2022.xlsx')
files.download('Eilat_matnasim_with_stat_2022.csv')

✓ Files saved. Downloading now...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>